<a href="https://www.kaggle.com/code/emigiupponi/btc-twitter-data-api-tweepy?scriptVersionId=109967082" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

https://datascienceparichay.com/article/python-get-data-from-twitter-api-v2/

# BTC Twitter data (API & Tweepy)

# Setup

In [1]:
pip install tweepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.2/101.2 kB 1.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import tweepy

# Authenticate

In [3]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret('TWITTER_BEARER')

In [4]:
client = tweepy.Client(bearer_token = secret_value_0)
client

# Collect Tweets

In [5]:
query = "bitcoin"

In [6]:
# your start and end time for fetching tweets
start_time = "2022-10-01T00:00:00Z"
end_time = "2022-10-02T00:00:00Z"

In [7]:
tweets = client.search_all_tweets(query=query,
                                  start_time=start_time,
                                  end_time=end_time,
                                  tweet_fields = ["created_at", "text"],
                                  user_fields = ["name", "username", "location"],
                                  place_fields=["full_name", "id", "country", "country_code", "geo", "name", "place_type"],
                                  expansions="geo.place_id,author_id",
                                  max_results = 100,
                                 )
#https://docs.tweepy.org/en/stable/client.html#tweepy.Client.search_recent_tweets

The expansions parameter is used to indicate that we are requesting additional information, for example, user_fields in the above code. The expanded object metadata will be returned with the “includes” response object.
Here tweets.data contains the tweet related data while tweets.includes["users] contains the additional user data that we requested. Let’s look at this information for the first tweet (present at index 0).
https://datascienceparichay.com/article/python-get-data-from-twitter-api-v2/

# Create a dataset


In [8]:
# import the pandas library
import pandas as pd
# create a list of records
tweet_info_ls = []
# iterate over each tweet and corresponding user details
for tweet, user in zip(tweets.data, tweets.includes['users']):
    tweet_info = {
        'tweet_id': tweet.id,
        'created_at': tweet.created_at,
        'in_reply_to_user_id': tweet.in_reply_to_user_id,
        'text': tweet.text,
        'user_id': tweet.author_id,
        'name': user.name,
        'username': user.username,
        'location': user.location,
        'verified': user.verified,
        'description': user.description
    }
    tweet_info_ls.append(tweet_info)
# create dataframe from the extracted records
tweets_df = pd.DataFrame(tweet_info_ls)
# display the dataframe
tweets_df.head(6)

#https://youtu.be/rQEsIs9LERM

,tweet_id,created_at,in_reply_to_user_id,text,user_id,name,username,location,verified,description
0,1576361313538052096,2022-10-01 23:59:59+00:00,None,@MasonVersluis Do well to understand that ever...,338972747,Keith Hilderbrand Sr.,Keith_Hilder,"Ohio, USA",None,None
1,1576361307971862528,2022-10-01 23:59:57+00:00,None,"RT @Csr_Dpr_BTC: 目先は19,300をサポレジするのか？どうかに着目していて...",804577546872336384,ビットコインのつぶやき,bitcoin_sougo,None,None,None
2,1576361300833230849,2022-10-01 23:59:56+00:00,None,RT @ChainSnake: https://t.co/9azTezog2E regist...,859863589842280449,Bidmaster777,bidmaster777,None,None,None
3,1576361296034930688,2022-10-01 23:59:55+00:00,None,Bitcoin price index https://t.co/o7UcHJUhC6 #U...,2395138046,WorldCoinIndex,WorldCoinIndex,None,None,None
4,1576361294466195461,2022-10-01 23:59:54+00:00,None,RT @nayibbukele: Shots fired! 👀\n\n#Bitcoin🇸🇻,1447614767540101122,Melvin Orellana,MelvinRo_,None,None,None
5,1576361290200584193,2022-10-01 23:59:53+00:00,None,RT @seth_fin: 💵💵💵🌱🌱🌱\n#BITCOIN have another re...,1576197725841854465,bingebunny,bingebunnie,None,None,None


In [9]:
tweets_df.to_csv('tweets_df.csv', index=None, sep=' ', mode='a')